In [10]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [15]:
#Revisión general de los datos:
import ast
import pandas as pd

rows = []
with open("steam_games.json") as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

df = pd.DataFrame(rows)

In [16]:
# Convertir la columna "release_date" a formato de fecha y extraer el año
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['year'] = df['release_date'].dt.year

# Codificar la variable categórica "genre"
label_encoder = LabelEncoder()
df = df.dropna(subset=['genres'])  # Eliminar filas con valores 'float' en la columna 'genres'
df['genres'] = df['genres'].apply(lambda x: ', '.join(x))  # Convertir listas a cadenas
df['genres_encoded'] = label_encoder.fit_transform(df['genres'])
df.drop('genres', axis=1, inplace=True)

# Convertir la columna 'metascore' a valores numéricos y reemplazar los valores 'nan' y 'na' por NaN
df['metascore'] = pd.to_numeric(df['metascore'], errors='coerce')

# Tratar los valores 'Free to Play' en la columna 'price' como 0
df.loc[df['price'] == 'Free to Play', 'price'] = 0

# Convertir la columna 'price' a valores numéricos y reemplazar los valores 'nan' por NaN
df['price'] = pd.to_numeric(df['price'], errors='coerce')

# Eliminar filas que contienen NaN en cualquier columna excepto 'price'
df.dropna(subset=['genres_encoded', 'year', 'metascore'], inplace=True)

# Eliminar filas que contienen NaN en la columna 'price'
df.dropna(subset=['price'], inplace=True)

# Seleccionar las características para el modelo
X = df[['genres_encoded', 'year', 'metascore']]
y = df['price']  # Variable objetivo que deseamos predecir (precio)

# Aplicar SimpleImputer para reemplazar los valores NaN en las características restantes
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

# Entrenar el modelo de regresión
model = LinearRegression()
model.fit(X_train, y_train)

C:\Users\naomi\AppData\Local\Temp\ipykernel_20812\1131334403.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')


LinearRegression()

In [17]:
# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

In [18]:
# Calcular el error cuadrático medio (RMC)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Error Cuadrático Medio (RMC):", rmse)

Error Cuadrático Medio (RMC): 9.776057595629865
